# Weapons and Armor Scraping

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [239]:
def extract_links(http):
    table_name = http.split('.com/')[1].split('.')[0]
    prefix = http.split('/'+table_name)[0]+'/'
    webpage = requests.get(http)
    soup = BeautifulSoup(webpage.content, "html.parser")
    all_links = soup.find_all('a')
    links = []
    cleaner = ""
    if "Category" in http:
        cleaner = "Display"
    if "Proficiency" in http:
        cleaner = "Display"
        table_name = "Weapon"
    if "Feats" in http:
        table_name = "FeatDisplay"
    for link in all_links:
        if table_name+cleaner in link['href'] and 'javascript:' not in link['href'] and '=Rules' not in link['href'] and '=All' not in link['href'] and "Category=Equipment" not in link['href']:
            links.append(prefix+link["href"])
    print(links)
    return links[150:]

def clean_links(links, string):
    links = [link for link in links if string in link]
    return links

def links_to_df(links, nb_col):
    maper = [2*i for i in range(nb_col+1)]
    labels = []
    rows = []
    for link in links:
        table_name = link.split('.com/')[1].split('.')[0]
        if '&Family=' in link:
            family_name = link.split('&Family=')[1].strip()
            item_name = link.split('&Family=')[0].split('ItemName=')[1].strip()
            print(table_name, family_name, item_name)
        else:
            family_name = 'None'
        webpage = requests.get(link)
        soup = BeautifulSoup(webpage.content, "html.parser")
        if "Display" in link:
            if "OtherItems" in link:
                targets = soup.find_all(id = re.compile(f"ctl00_MainContent_DataListOtherItems_ctl0\d_LabelName"))
            elif "Feat" in link:
                targets = soup.find_all(id = re.compile(f"ctl00_MainContent_DataListTypes_ctl0\d_LabelName"))
            else:
                targets = soup.find_all(id = re.compile(f"ctl00_MainContent_DataListTalentsAll_ctl0\d_LabelName"))
        else:
            targets = soup.find_all(id = re.compile(f"ctl00_MainContent_DataList{table_name}_ctl0\d_LabelName"))
        data = ''.join([target.get_text("|") for target in targets])
        data = '|'.join(data.split('| |'))
        if family_name == 'None':
            fish = data.split('|')
            if "Feat" in link:
                fish = fish[:3]+['Benefit']+fish[3:]
            description = ''.join(fish[(2*nb_col-3):])
        else:
            if family_name == item_name:
                full_name = item_name
                family_description = data.split(full_name)[1]
                unnamed_data = data.split(full_name)[2]
            else:
                full_name = family_name + ', ' + item_name
                family_description = data.split(full_name)[0]
                if len(data.split(full_name)) > 1:
                    unnamed_data = data.split(full_name)[1]
                else:
                    print("full name not in fish")
                    unnamed_data = family_name
            data = full_name + unnamed_data.split(family_name)[0]
            fish = data.split('|')
            if len(fish) > 2*nb_col-2:
                description = ''.join(fish[(2*nb_col-2):])+ family_description
            else:
                description = family_description
        if len(fish) >= 2*nb_col-3:
            fish = ['Name']+fish[:(2*nb_col-3)]+['Description']+[description]+['Nethys']+[link]
            row = [fish[i+1].strip() for i in maper]
            rows.append(row)
            if labels == []:
                labels= [fish[i].strip() for i in maper]
        else:
            internal_link = link.split('.com/')[1]
            target = soup.select_one(f'a[href="{internal_link}"]')
            bowl = target.next_elements
            fishy = []
            print("Brute fishing:")
            for string in bowl:
                if '<' not in repr(string) and repr(string) != "' '":
                    fishy.append(string)
                    if len(fishy) == 2*nb_col:
                        break
            fishy = [fish.strip() for fish in fishy]
            if len(fishy) < 2*nb_col-1:
                fishy += ['?' for i in range(len(fishy), 2*nb_col)]
            if description == '':
                description = ''.join(fishy[2*nb_col-1:])
            fish = ['Name']+fishy[:(2*nb_col-3)]+['Description']+[description]+['Nethys']+[link]
            row = [fish[i+1].strip() for i in maper]
            rows.append(row)
            if labels == []:
                safety_labels = [fish[i].strip() for i in maper]
        print(row[0])
    if labels == []:
        labels = safety_labels
    df = pd.DataFrame(rows,columns=labels)
    return df

def link_to_csv(link, nb_col, name='New'):
    raw_links = extract_links(link)
    df = links_to_df(raw_links, nb_col)
    df.to_csv('Raw_csv/' + name + '_Raw.csv',sep='|',index=False)
    return


In [181]:
webpage = requests.get('https://aonsrd.com/MagicItems.aspx?ItemName=Cloudy Blue Rhomboid&Family=Aeon Stone')
soup = BeautifulSoup(webpage.content, "html.parser")
target = soup.select_one('a[href="MagicItems.aspx?ItemName=Cloudy Blue Rhomboid&Family=Aeon Stone"]')
bowl = target.next_elements
fishs = []
nb_col = 6
for string in bowl:
    if '<' not in repr(string) and repr(string) != "' '":
        print(repr(string).strip())
        print(len(repr(string).strip()))
        fishs.append(string)
        if len(fishs) == 2*nb_col-1:
            break
fishs = [fish.strip() for fish in fishs]
print(fishs)   
    
 #   print(element.text)

' Aeon Stone, Cloudy Blue Rhomboid'
35
'Source'
8
'Starfinder #8: Escape from the Prison Moon pg. 44'
51
'Level'
7
' 3; '
6
'Price'
7
' 1,300; '
10
'Bulk'
6
' —'
4
'Description'
13
'While this '
13
['Aeon Stone, Cloudy Blue Rhomboid', 'Source', 'Starfinder #8: Escape from the Prison Moon pg. 44', 'Level', '3;', 'Price', '1,300;', 'Bulk', '—', 'Description', 'While this']


HTTP links :

In [ ]:
#light_armors = 'https://aonsrd.com/Armor.aspx?Category=Light'
#heavy_armors = 'https://aonsrd.com/Armor.aspx?Category=Heavy'
#powered_armors = 'https://aonsrd.com/PoweredArmor.aspx?ItemName=All'
#armor_upgrades = 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=All&Family=None'

#biotech = 'https://www.aonsrd.com/Biotech.aspx?ItemName=All&Family=None'
#cybernetics = 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=All&Family=None'
#magitech = 'https://www.aonsrd.com/Magitech.aspx?ItemName=All&Family=None'
#necrografts = 'https://www.aonsrd.com/Necrografts.aspx?ItemName=All&Family=None'

#hybriditems = 'https://www.aonsrd.com/HybridItems.aspx?ItemName=All&Family=None'
#magicitems = 'https://www.aonsrd.com/MagicItems.aspx?ItemName=All&Family=None'
#techitems = 'https://www.aonsrd.com/TechItems.aspx?ItemName=All&Family=None'

#longarms_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Longarms'
#smallarms_wp ='https://www.aonsrd.com/Weapons.aspx?Proficiency=SmallArms'
#sniper_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Sniper'
#heavy_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Heavy'
#advanced_melee_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=AdvMelee'
#basic_melee_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=BasicMelee'
#grenades_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Grenade'
secial_wp = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Special'

#solarian = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Solarian'
#weapon_accessories = 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=All&Family=None'
#weapon_fusions = 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=All'
#ammo = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Ammo'


Armors

In [22]:
link = 'https://aonsrd.com/Armor.aspx?Category=Light'
link_to_csv(link, 13, 'Light_Armors')

['https://aonsrd.com/ArmorDisplay.aspx?ItemName=Flight Suit&Family=Stationwear', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Recruit&Family=Shobhad Harness', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Utility&Family=EJ Coverall', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Nascent&Family=Flight Skin', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Second Skin&Family=None', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Estex Suit', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Kasatha Microcord', "https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Perserver's Mantle", 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Zeizerer Diffractor', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Freebooter Armor', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Industrial&Family=EJ Coverall', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Vesk Brigandine', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Hardened Resin&Family=None', 'https://a

Stationwear, Flight Suit
Shobhad Harness, Recruit
EJ Coverall, Utility
Flight Skin, Nascent
Second Skin
Estex Suit, I
Kasatha Microcord, I
Perserver's Mantle, I
Zeizerer Diffractor, I
Freebooter Armor, I
EJ Coverall, Industrial
Vesk Brigandine, I
Hardened Resin
Skitterhide, I
Carbon Skin, Graphite
Stationwear, Casual
Clearweave, I
Regimental Dress, I
Shobhad Harness, Veteran
Lashunta Tempweave, Basic
Peacekeeper’s Aegis, I
Acrochor Hide, Basic
Grave Mantle, Enlisted
Defrex Hide, Basic
Stationwear, Business
Vesk Brigandine, II
Estex Suit, II
Plexigrass Bodysuit, I
D-Suit, I
Flight Skin, Foray
Kasatha Microcord, II
Stationwear, Elite
Ysoki Refractor Suit
Zeizerer Diffractor, II
Polyplate, Basic
Lashunta Mind Mail, I
Freebooter Armor, II
Dust Manta Hide, Basic
Estex Suit, III
Perserver's Mantle, II
D-Suit, II
AbadarCorp Travel Suit, Silver
Aeon Guard, SpecOps Armor
Azlanti Royal Battle Regalia
Peacekeeper’s Aegis, II
Acrochor Hide, Advanced
Lashunta Tempweave, Advanced
Shobhad Harness, Wa

In [15]:
link = 'https://aonsrd.com/Armor.aspx?Category=Heavy'
link_to_csv(link, 13, 'Heavy_Armors')

['https://aonsrd.com/ArmorDisplay.aspx?ItemName=Troop&Family=Ceremonial Plate', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Reinforced EVA Suit', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Golemforged Plating', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Utility&Family=EJ Hardsuit', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Lashunta Ringwear', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Hidden Soldier Armor&Family=None', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Worker&Family=Formian Plate', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Basic&Family=Iridishell', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Kyokor Plating', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=I&Family=Mining Jack', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Armiger&Family=Hellknight Plate', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Thinplate&Family=None', 'https://aonsrd.com/ArmorDisplay.aspx?ItemName=Industrial&Family=EJ Hardsuit', 'https

Defiance Series, Elite
Dendron Armor, Yew
Ceremonial Plate, Commander
Lashunta Ringwear, III
Night Plate, I
Terramold Plate, I
Vesk Overplate, II
Hardplate, Advanced
Kalo Encounter Suit, II
Iridishell, Advanced
Skittersuit, I
EJ Hardsuit, Elite
Skyfire Armor, Pinion
Hellknight Plate, Maralictor
Formian Hide, Superior
Defiance Series, Specialist
Kyokor Plating, III
Mining Jack, III
Dendron Armor, Oak
Aeon Guard, Battle Dress (Command)
Vesk Overplate, III
Enforcer Armor, II
Golemforged Plating, IV
Lashunta Ringwear, IV
Night Plate, II
Terramold Plate, II
Vesk Monolith, I
Formian Plate, Taskmaster
Iridishell, Superior
Aegis Series, Squad
Hardplate, Elite
Kalo Encounter Suit, III
Skittersuit, II
EJ Hardsuit, Paragon
Skyfire Armor, Exident
Fossilwrap, II
Golemforged Plating, V
Inheritor's Grace, II
Vesk Overplate, IV
Kyokor Plating, IV
Hellknight Plate, Paralictor
Lashunta Ringwear, V
Enforcer Armor, III
Enginerunner
Steelbones
Terramold Plate, III
Vesk Monolith, II
Night Plate, III
Aegis S

In [44]:
link = 'https://aonsrd.com/PoweredArmor.aspx?ItemName=All'
link_to_csv(link, 18, 'Powered_Armors')


['https://aonsrd.com/PoweredArmor.aspx?ItemName=Goblin Battle Jack', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Cargo Lifter', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Encounter Suit', "https://aonsrd.com/PoweredArmor.aspx?ItemName=Scrapper's Rig", 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Absorptive Shell', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Battle Harness', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Spacer Carapace', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Laborer Frame', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Personal Submersible', "https://aonsrd.com/PoweredArmor.aspx?ItemName=Spellcaster's Aegis", 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Brawler Frame', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Hover Suit', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Ironclad Bulwark', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Spider Harness', 'https://aonsrd.com/PoweredArmor.aspx?ItemName=Flight Frame', 'https://aonsrd.com/PoweredAr

In [42]:
link = 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=All&Family=None'
link_to_csv(link, 8, 'Armor_Upgrades')

['https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Weapon Spikes&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Pet Carrier&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Remote Monitoring System&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Easy Access Kit&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Grim Trophies&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Light Projector&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Tensile Reinforcement&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Thermal Regulator&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Basic&Family=Sustenance System', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Concealed Compartments&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Aversion Coating&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?ItemName=Infrared Sensors&Family=None', 'https://aonsrd.com/ArmorUpgrades.aspx?Ite

Weapon Spikes
Pet Carrier
Remote Monitoring System
Easy Access Kit
Grim Trophies (Magic)
Light Projector
Tensile Reinforcement
Thermal Regulator (Hybrid)
Sustenance System, Basic
Concealed Compartments
Aversion Coating
Infrared Sensors
Radiation Buffer
Leapers
Enviro-Sensor
Voice-Command Module, Mk 1
Slickskin
Quick-Release Sheath
Agility Enhancer, Mk 1
Stabilizer Springs
Descent Thrusters
Snarl Barbs
Automated Loader
Sustenance System, Mk 1
Auto-CPR Unit
Jump Jets
Omnidirectional Camera
Thrower Arms
Juggernaut Boosters
Quicksuit (Hybrid)
Glamer Projector (Magic)
Voice-Command Module, Mk 2
Hydrojet
Hibernation System
Mobility Enhancer, Mk 1
Reactive Polarization
Force Field, Brown
Exit Pod
Remote-Link Module
Hover Field
Climate Control Unit
Flashblinders, MK 1
Backup Generator
Longstrider Module
Sustenance System, Mk 2
Targeting Computer
Load Lifter
Telepathic Dampener (Magic)
Voice-Command Module, Mk 3
Rampart Plates
Ghost Armor (Magic)
Agility Enhancer, Mk 2
Electrostatic Field, MK 1

Weapons

In [52]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Longarms'
#extract_links(link)
link_to_csv(link, 14, 'Longarms_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Needler Rifle&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Liquidator&Family=Caustolance', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Needler Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Pulse&Family=Rackarack', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Formian Venomcaster', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Growl-class&Family=Frost Maw', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Advanced&Family=Needler Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Decimator&Family=Caustolance', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Surge&Family=Rackarack', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Growl-class&Family=Chimera Graft', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Recombinator Ray&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Advanced&Family=Formian Venomcaster

Needler Rifle
Caustolance, Liquidator
Needler Rifle, Tactical
Rackarack, Pulse
Formian Venomcaster, Tactical
Frost Maw, Growl-class
Needler Rifle, Advanced
Caustolance, Decimator
Rackarack, Surge
Chimera Graft, Growl-class
Recombinator Ray
Formian Venomcaster, Advanced
Frost Maw, Snarl-class
Nanite Thrower, Tactical
Reality Rifle
Frost Maw, Roar-class
Needler Rifle, Elite
Chimera Graft, Snarl-class
Caustolance, Executioner
Nanite Thrower, Advanced
Rackarack, Drum
Formian Venomcaster, Elite
Nanite Thrower, Elite
Needler Rifle, Paragon
Chimera Graft, Roar-class
Caustolance, Eradicator
Formian Venomcaster, Paragon
Rackarack, Hammer
Nanite Thrower, Paragon
Needler Rifle, Supreme
Numbing Beam, Tactical
Ice Carbine, Subzero
Void Rifle, Grave-Class
Frailty Rifle, Atrophy-Class
Freeze Ray, Hiemal
Zero Rifle, Frostbite-Class
Numbing Beam, Advanced
Frost Projector, Frostbite-Class
Void Rifle, Crypt-Class
Frailty Rifle, Rot-Class
Freeze Ray, Algid
Zero Rifle, Hailstorm-Class
Ice Carbine, Gelid
Nu

In [53]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=SmallArms'
link_to_csv(link, 14, 'Smallarms_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Needler Pistol&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Liquidator&Family=Caustoject', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Cestus Pistol', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Needler Pistol', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Skipshot Pistol', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Glove Needler', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Micrergate&Family=Acid Cannon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Linear&Family=Graviton Pistol', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Ifrit-Class&Family=Infernian', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Nightarch Needler', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Crescent&Family=Moon Splinter', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Sentry&Family=Shield 

Needler Pistol
Caustoject, Liquidator
Cestus Pistol, Tactical
Needler Pistol, Tactical
Skipshot Pistol, Tactical
Glove Needler, Tactical
Acid Cannon, Micrergate
Graviton Pistol, Linear
Infernian, Ifrit-Class
Nightarch Needler, Tactical
Moon Splinter, Crescent
Shield Projector, Sentry
Sting Pistol, Ant
Neutriad Pistol
Binding Blaster, Orb
Graviton Pistol, Vector
Needler Pistol, Advanced
Caustoject, Decimator
Cestus Pistol, Advanced
Skipshot Pistol, Advanced
Reality Pistol
Acid Cannon, Macrergate
Glove Needler, Advanced
Infernian, Salamander-Class
Tetrad Rings, Tactical
Graviton Pistol, Tensor
Nightarch Needler, Advanced
Binding Blaster, Globe
Moon Splinter, Gibbous
Shield Projector, Defender
Moon Splinter, Full
Moon Splinter, New
Needler Pistol, Elite
Sting Pistol, Yellow Jacket
Cestus Pistol, Elite
Infernian, Hellhound-Class
Acid Cannon, Dinergate
Caustoject, Executioner
Skipshot Pistol, Elite
Tetrad Rings, Advanced
Graviton Pistol, Chiral
Nightarch Needler, Elite
Glove Needler, Elite


In [54]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Sniper'
link_to_csv(link, 14, 'Sniper_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Yellow Jacket&Family=Wraith-Sting Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Kin-Killer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Wasp&Family=Wraith-Sting Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Nanite Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Advanced&Family=Kin-Killer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Hornet&Family=Wraith-Sting Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Elite&Family=Kin-Killer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Advanced&Family=Nanite Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Elite&Family=Nanite Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Paragon&Family=Kin-Killer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Terrestrial&Family=Gamma Rifle', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Ionizing&Family=Gamma Rifle', 

In [55]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Heavy'
link_to_csv(link, 14, 'Heavy_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Merc&Family=NIL Grenade Launcher', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Utility &Family=Molecular Borer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=I&Family=Psychic-Wave Cannon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Merc&Family=Minelayer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Corroder-Class&Family=Acid Lancer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Rage-Bringer&Family=Desperation Cannon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Sheet&Family=Electrogel Jet', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=II&Family=Psychic-Wave Cannon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Cluster Launcher', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Industrial &Family=Molecular Borer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Ribbon&Family=Electrogel Jet', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Squad&Fam

NIL Grenade Launcher, Merc
Molecular Borer, Utility
Psychic-Wave Cannon, I
Minelayer, Merc
Acid Lancer, Corroder-Class
Desperation Cannon, Rage-Bringer
Electrogel Jet, Sheet
Psychic-Wave Cannon, II
Cluster Launcher, Tactical
Molecular Borer, Industrial
Electrogel Jet, Ribbon
Minelayer, Squad
Dissolver, Macrergate
NIL Grenade Launcher, Squad
Acid Lancer, Melter-Class
IMDS Missile Launcher
Desperation Cannon, Doom-Caller
Molecular Borer, Advanced
Electrogel Jet, Rocket
Psychic-Wave Cannon, III
Cluster Launcher, Advanced
Dissolver, Dinergate
Minelayer, Platoon
Acid Lancer, Liquefier-Class
Molecular Borer, Elite
Cluster Launcher, Elite
Desperation Cannon, Sky-Piercer
Electrogel Jet, Smooth-channel
Psychic-Wave Cannon, IV
Minelayer, Command
Cluster Launcher, Paragon
Dissolver, Ergatoid
Molecular Borer, Paragon
Acid Lancer, Disintegrator-Class
Desperation Cannon, Titan-Killer
Psychic-Wave Cannon, V
Hailcannon, Subzero
Ice Launcher, Aufeis
Coolant Sprayer, Hiemal
Energy Converter, Tactical
Ic

In [58]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=AdvMelee'
link_to_csv(link, 11, 'Advanced_Melee_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Assault&Family=Hammer', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Standard&Family=Longsword', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Starknife', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Standard&Family=Taclash', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Injection Glove&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Standard&Family=Needler Estoc', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Ascetic&Family=Velstrac Flenser', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Hook Sword&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Ghoulish&Family=Painclaw', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Drone&Family=Talon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Warfan&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Battleaxe', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName

Hammer, Assault
Longsword, Standard
Starknife, Tactical
Taclash, Standard
Injection Glove
Needler Estoc, Standard
Velstrac Flenser, Ascetic
Hook Sword
Painclaw, Ghoulish
Talon, Drone
Warfan
Battleaxe, Tactical
Hardlight Hand Wrap, Diffuse
Thresherblade, Tactical
Living Lash
Doshakari, Tactical
Mindspike, Tactical
Needler Estoc, Sharp-pointed
Fangblade
Longsword, Sintered
Polylash, Tactical
Velstrac Flenser, Singer
Starknife, Sintered
Talon, Warrior
Hardlight Hand Wrap, Ambient
Longsword, Microserrated
Thresherblade, Microserrated
Doshakari, Advanced
Hammer, Comet
Mindspike, Microserrated
Needler Estoc, Hyper-pointed
Longsword, Ultrathin
Painclaw, Demonic
Polylash, Advanced
Taclash, Numbing
Starknife, Accelerated
Talon, Knight
Velstrac Flenser, Debater
Grindblade
Doshakari, Ultrathin
Thresherblade, Nanoserrated
Hardlight Hand Wrap, Natural
Longsword, Zero-Edge
Mindspike, Nanoserrated
Needler Estoc, Ultra-pointed
Monowhip
Polylash, Elite
Doshakari, Zero-Edge
Hammer, Meteoric
Starknife, L

Singing Spear, Sopranino
Seismic Pick, Heavy
Interference Blade, Monophonic
Vibrogarrote, Harmonic
Singing Spear, Soprano
Seismic Pick, Driver
Interference Blade, Harmonic
Vibrogarrote, Interference
Singing Spear, Alto
Resonant Staff, Reverberant
Interference Blade, Polyphonic
Seismic Pick, Demolition
Vibrogarrote, Infrasonic
Singing Spear, Tenor
Resonant Staff, Stentorian
Interference Blade, Multiphonic
Vibrogarrote, Ultrasonic


In [59]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=BasicMelee'
link_to_csv(link, 11, 'Basic_Melee_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Unarmed Strike&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Club&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Baton', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Cestus&Family=Battleglove', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Basic&Family=Handaxe', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Survival&Family=Knife', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Sword Cane', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Standard&Family=Syringe Stick', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Idaran Force Training Baton&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Austere&Family=Bone Cestus', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Tactical&Family=Dueling Sword', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Jolt &Family=Electric Prod', 'https://www.aonsrd.com/WeaponDispl

Unarmed Strike
Club
Baton, Tactical
Battleglove, Cestus
Handaxe, Basic
Knife, Survival
Sword Cane, Tactical
Syringe Stick, Standard
Idaran Force Training Baton
Bone Cestus, Austere
Dueling Sword, Tactical
Electric Prod, Jolt
Sap, Light
Sledge
Switchblade, Tactical
Inferno Knife
Tailblade
Dueling Sword (Corpse Fleet), Converted
Handaxe, Tactical
Sword Cane, Advanced
Retractable Spike, Tactical
Spined Blade, Basic
Syringe Stick, Sharp-pointed
Electric Prod, Impulse
Polyclub, Tactical
Retractable Stinger, Tactical
Sap, Medium
Shell Knuckles, Tactical
Hook Knife
Bone Cestus, Measured
Knife, Tactical
Sword Cane, Ultrathin
Dueling Sword, Buzzblade
Spined Blade, Buzzblade
Switchblade, Ultrathin
Incapacitator
Handaxe, Sintered
Retractable Stinger, Advanced
Syringe Stick, Hyper-pointed
Battleglove, Power
Dueling Sword (Corpse Fleet), Officer
Electric Prod, Storm
Polyclub, Advanced
Retractable Spike, Advanced
Sword Cane, Zero-Edge
Dueling Sword, Ultrathin
Sap, Heavy
Spined Blade, Ripper
Bone Ces

In [79]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Grenade'
link_to_csv(link, 13, 'Grenade_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Smoke Grenade&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Frag Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Holo Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Shock Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Stickybomb Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=I&Family=Decoupler Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Flash Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Holy Water Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Incendiary Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Riot Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Diminisher Grenade', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mk 1&Family=Foam Grenade', 'https://www.aonsrd.co

Smoke Grenade
Frag Grenade, Mk 1
Holo Grenade, Mk 1 (Hybrid)
Shock Grenade, Mk 1
Stickybomb Grenade, Mk 1
Decoupler Grenade, I
Flash Grenade, Mk 1
Holy Water Grenade, Mk 1
Incendiary Grenade, Mk 1
Riot Grenade, Mk 1
Diminisher Grenade, Mk 1 (Hybrid)
Foam Grenade, Mk 1
Solar Flare Grenade, I (Hybrid)
Summoning Grenade, Mk 1 (Hybrid)
Web Grenade, Mk 1
Taserweb Grenade
Frag Grenade, Mk 2
Holo Grenade, Mk 2 (Hybrid)
Pheromone Grenade, Mk 1
Screamer Grenade, Mk 1
Shock Grenade, Mk 2
Stickybomb Grenade, Mk 2
Holy Water Grenade, Mk 2
Necro Grenade, Mk 1
Pulse Grenade, Mk 1
Riot Grenade, Mk 2
Cryo Grenade, Mk 1
Decoupler Grenade, II
Diminisher Grenade, Mk 2 (Hybrid)
Flash Grenade, Mk 2
Incendiary Grenade, Mk 2
Microbot Grenade, Mk 1 (Hybrid)
Solar Flare Grenade, II (Hybrid)
Summoning Grenade, Mk 2 (Hybrid)
Holo Grenade, Mk 3 (Hybrid)
Riot Grenade, Mk 3
Disruption Grenade
Frag Grenade, Mk 3
Incendiary Grenade, Mk 3
Pheromone Grenade, Mk 2
Screamer Grenade, Mk 2
Junkbot Grenade (Hybrid)
Diminish

In [ ]:
Need to split links 1H vs 2H
#link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Special'
#link_to_csv(link, 14, 'Special_Weapons')

In [80]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Solarian'
link_to_csv(link, 10, 'Solarian_Weapons')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=Apocalypse Crystal', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=Electron', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=Photino Crystals', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=Photon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Least&Family=Tauon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=W-Boson', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=Z-Boson', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Shard&Family=Gluon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Least&Family=Muon', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Least&Family=T-Quark', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Least&Family=Electron', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Least&Family=Positron', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Minor&Family=Muon', 'https://www

In [81]:
link = 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=All&Family=None'
link_to_csv(link, 7, 'Weapon_Accessories')

['https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Grenadier Bracket&Family=None', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Rangefinder&Family=None', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Uniclamp&Family=None', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Heavy&Family=Bayonet Bracket', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Light&Family=Bayonet Bracket', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Light&Family=Bipod', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Heavy&Family=Bipod', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Folding&Family=Grip', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Nanofiber&Family=Grip', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Manual&Family=Sight', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Collapsing&Family=Stock', 'https://www.aonsrd.com/WeaponAccessories.aspx?ItemName=Collapsing Weapon&Family=None', 'https://www.aon

In [82]:
link = 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=All'
link_to_csv(link, 4, 'Weapon_Fusions')

['https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Anchoring', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Called', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Defiant', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Durable', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Glamered', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Hindering', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Hybridized', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Illuminating', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Invigorating', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Ominous', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Returning', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Secured', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Soulfire', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Stabilizing', 'https://www.aonsrd.com/WeaponFusions.aspx?ItemName=Trailblazer', 'https://www.aonsrd.com/WeaponFusions.asp

In [207]:
link = 'https://www.aonsrd.com/Weapons.aspx?Proficiency=Ammo'
link_to_csv(link, 12, 'Ammunitions2')

['https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Arrows&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Darts&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Flare&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Flechette&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Scattergun Shells&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Standard&Family=Battery', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Standard&Family=Petrol Tank', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Longarm and Sniper&Family=Rounds', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Small Arm&Family=Rounds', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Heavy&Family=Rounds', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=High-Capacity&Family=Petrol Tank', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=Mini-Rockets&Family=None', 'https://www.aonsrd.com/WeaponDisplay.aspx?ItemName=High-

Augmentations

In [98]:
link = 'https://www.aonsrd.com/Biotech.aspx?ItemName=All&Family=None'
link_to_csv(link, 6, 'Augmentations_Biotech')

['https://www.aonsrd.com/Biotech.aspx?ItemName=Gill Sheath&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Pressurized Lungs&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Restraining Spinneret&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Wildwise&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Photoenergetic Node&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Venom Spur&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Mk 1&Family=Resistant Hide', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Wyrmling&Family=Dragon Gland', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Swimming Fins&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Adaptive Biochains&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Climbing Suckers&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Mighty Vocal Cords&Family=None', 'https://www.aonsrd.com/Biotech.aspx?ItemName=Clinging Hands&Family=None', 'https://ww

In [99]:
link = 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=All&Family=None'
link_to_csv(link, 6, 'Augmentations_Cybernetics')

['https://www.aonsrd.com/Cybernetics.aspx?ItemName=Standard&Family=Prosthetic Limb', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Anosmatic Shunt&Family=None', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Vocal Modulator&Family=None', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Voice Amplifier&Family=None', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Standard&Family=Hideaway Limb', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Single&Family=Society Subdermal Graft', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Respiration Compounder&Family=None', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Lightvision Shades&Family=None', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Standard&Family=Datajack', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Minimal&Family=Spinal Struts', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Retinal Reflectors&Family=None', 'https://www.aonsrd.com/Cybernetics.aspx?ItemName=Storage&Family=Prosthetic Limb', 'https://

In [102]:
link = 'https://www.aonsrd.com/Magitech.aspx?ItemName=All&Family=None'
link_to_csv(link, 6, 'Augmentations_Magitech')

['https://www.aonsrd.com/Magitech.aspx?ItemName=Technopathy Node&Family=None', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Basic&Family=Morphic Skin', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Mk 1&Family=Gravitational Harness', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Fascinating&Family=Psychoactive Eyes', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Follower&Family=Velstrac Shackle', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Mk 1&Family=Intercepting Ears', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Nimble Soles&Family=None', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Advanced&Family=Morphic Skin', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Mk 1&Family=Force Soles', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Selective Ears&Family=None', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Charged Skin&Family=None', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Mk 1&Family=Rune-Etched Chitin', 'https://www.aonsrd.com/Magitech.aspx?ItemName=Charming&Famil

In [104]:
link = 'https://www.aonsrd.com/Necrografts.aspx?ItemName=All&Family=None'
link_to_csv(link, 3, 'Augmentations_Necrografts')

['https://www.aonsrd.com/Necrografts.aspx?ItemName=Black Heart&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Bone Blade&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Corpseskin&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Crypt Marrow&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Enervating Hand&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Ghoul Glands&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Grave Wind&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Keening Larynx&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Moonlight Fibers&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Necrocortex&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Phantom Basal Ganglia&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Shadow Nerves&Family=None', 'https://www.aonsrd.com/Necrografts.aspx?ItemName=Squirming Entr

In [ ]:
link = 
personal_upgrades

Main Items

In [205]:
link = 'https://www.aonsrd.com/HybridItems.aspx?ItemName=All&Family=None'
link_to_csv(link, 6, 'Hybrid_Items')

['https://www.aonsrd.com/HybridItems.aspx?ItemName=Attablossom&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Dust Goggles&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Instant Ground&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Plant Shield Amulet&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Shotalashu Saddle&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Starstone Compass&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Thermal Regulator&Family=None', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Tier 1&Family=Software Imp', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Portrait&Family=Empathic Depiction', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Silver&Family=Metal Nitro', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Mk 1&Family=Mindlink Circlet', 'https://www.aonsrd.com/HybridItems.aspx?ItemName=Lesser&Family=Shadowstaff', 'https://www.aonsrd.com/HybridItem

Brute fishing:
Attablossom
HybridItems None Dust Goggles
Brute fishing:
Dust Goggles
HybridItems None Instant Ground
Brute fishing:
Instant Ground
HybridItems None Plant Shield Amulet
Brute fishing:
Plant Shield Amulet
HybridItems None Shotalashu Saddle
Brute fishing:
Shotalashu Saddle
HybridItems None Starstone Compass
Brute fishing:
Starstone Compass
HybridItems None Thermal Regulator
Brute fishing:
Thermal Regulator
HybridItems Software Imp Tier 1
Brute fishing:
Software Imp, Tier 1
HybridItems Empathic Depiction Portrait
Brute fishing:
Empathic Depiction, Portrait
HybridItems Metal Nitro Silver
Brute fishing:
Metal Nitro, Silver
HybridItems Mindlink Circlet Mk 1
Brute fishing:
Mindlink Circlet, Mk 1
HybridItems Shadowstaff Lesser
Brute fishing:
Shadowstaff, Lesser
HybridItems None Book of Predilections
Brute fishing:
Book of Predilections
HybridItems None Deoxy Pellet
Brute fishing:
Deoxy Pellet
HybridItems None Intonation Pendant
Brute fishing:
Intonation Pendant
HybridItems None 

Brute fishing:
Software Imp, Tier 7
HybridItems Universal Solvent Forte
Brute fishing:
Universal Solvent, Forte
HybridItems None Dimensional Modulator
Brute fishing:
Dimensional Modulator
HybridItems None Entanglement Belt (Pair)
Brute fishing:
Entanglement Belt (Pair)
HybridItems None Quantum Processor
Brute fishing:
Quantum Processor
HybridItems Metal Nitro Star
Brute fishing:
Metal Nitro, Star
HybridItems Mindlink Circlet Mk 3
Brute fishing:
Mindlink Circlet, Mk 3
HybridItems Prescient Lenses Mk 3
Brute fishing:
Prescient Lenses, Mk 3
HybridItems Scrambler Glloves Mk 3
Brute fishing:
Scrambler Glloves, Mk 3
HybridItems None Erasure Worm
Brute fishing:
Erasure Worm
HybridItems Hivejack Mk 3
full name not in fish
Brute fishing:
Hivejack, Mk 3
HybridItems Mindshield Circlet Mk 3
full name not in fish
Brute fishing:
Mindshield Circlet, Mk 3
HybridItems Mnemonic Editor Mk 3
full name not in fish
Brute fishing:
Mnemonic Editor, Mk 3
HybridItems Retrieval Charms Mk 3
full name not in fish


In [199]:
link = 'https://www.aonsrd.com/MagicItems.aspx?ItemName=All&Family=None'
link_to_csv(link, 6, 'Magic_Items')

['https://www.aonsrd.com/MagicItems.aspx?ItemName=Clear Spindle&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Iridescent Spindle&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Cloudy Blue Rhomboid&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Opalescent White Pyramid&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Amber Hyperboloid&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Bright Yellow Cabochon&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Scarlet Prism&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Sapphire Cone&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Ruby Sphere&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Viridian Balbis&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=Purple Sphere&Family=Aeon Stone', 'https://www.aonsrd.com/MagicItems.aspx?ItemName=V

Aeon Stone, Clear Spindle
MagicItems Aeon Stone Iridescent Spindle
Aeon Stone, Iridescent Spindle
MagicItems Aeon Stone Cloudy Blue Rhomboid
Brute fishing:
Aeon Stone, Cloudy Blue Rhomboid
MagicItems Aeon Stone Opalescent White Pyramid
Brute fishing:
Aeon Stone, Opalescent White Pyramid
MagicItems Aeon Stone Amber Hyperboloid
Brute fishing:
Aeon Stone, Amber Hyperboloid
MagicItems Aeon Stone Bright Yellow Cabochon
Brute fishing:
Aeon Stone, Bright Yellow Cabochon
MagicItems Aeon Stone Scarlet Prism
Brute fishing:
Aeon Stone, Scarlet Prism
MagicItems Aeon Stone Sapphire Cone
Brute fishing:
Aeon Stone, Sapphire Cone
MagicItems Aeon Stone Ruby Sphere
Brute fishing:
Aeon Stone, Ruby Sphere
MagicItems Aeon Stone Viridian Balbis
Brute fishing:
Aeon Stone, Viridian Balbis
MagicItems Aeon Stone Purple Sphere
full name not in fish
Brute fishing:
Aeon Stone, Purple Sphere
MagicItems Aeon Stone Vibrant Green Prism
full name not in fish
Brute fishing:
Aeon Stone, Vibrant Green Prism
MagicItems Aeo

Brute fishing:
Rod of Cancellation
MagicItems Arclord's Rod Mk 1
Brute fishing:
Arclord's Rod, Mk 1
MagicItems Diffraction Cloak Mk 2
Brute fishing:
Diffraction Cloak, Mk 2
MagicItems Elemental Gem Mk 3
Brute fishing:
Elemental Gem, Mk 3
MagicItems Fiery Runeplates Mk 2
Brute fishing:
Fiery Runeplates, Mk 2
MagicItems Plasma Beads Mk 2
Brute fishing:
Plasma Beads, Mk 2
MagicItems Travel Treads Mk 2
Brute fishing:
Travel Treads, Mk 2
MagicItems None Shadow Orb
Brute fishing:
Shadow Orb
MagicItems None Starship in a Bottle
Brute fishing:
Starship in a Bottle
MagicItems None Temporal Crystal
Brute fishing:
Temporal Crystal
MagicItems None Aballonian Drone Box
Brute fishing:
Aballonian Drone Box
MagicItems None Incense of Open Thoughts
Brute fishing:
Incense of Open Thoughts
MagicItems None Inspiration Blazon
Brute fishing:
Inspiration Blazon
MagicItems Brenneri Stone Radiant
Brute fishing:
Brenneri Stone, Radiant
MagicItems None Mantle of Willpower
Brute fishing:
Mantle of Willpower
Magic

Spell Ampoule, 3rd
MagicItems Spell Gem 0
Spell Gem, 0
MagicItems Spell Gem 1st
Spell Gem, 1st
MagicItems Spell Gem 2nd
Spell Gem, 2nd
MagicItems Spell Gem 3rd
Spell Gem, 3rd
MagicItems Spell Gem 4th
Spell Gem, 4th
MagicItems Spell Gem 5th
Spell Gem, 5th
MagicItems Spell Gem 6th
Spell Gem, 6th
MagicItems Tiara of Translocation Mk 1
full name not in fish
Brute fishing:
Tiara of Translocation, Mk 1
MagicItems Tiara of Translocation Mk 2
full name not in fish
Brute fishing:
Tiara of Translocation, Mk 2
MagicItems Tiara of Translocation Mk 3
full name not in fish
Brute fishing:
Tiara of Translocation, Mk 3
MagicItems Tiara of Translocation Mk 4
Brute fishing:
Tiara of Translocation, Mk 4


In [113]:
link = 'https://www.aonsrd.com/TechItems.aspx?ItemName=All&Family=None'
link_to_csv(link, 6, 'Technological_Items')

['https://www.aonsrd.com/TechItems.aspx?ItemName=Adamantine Alloy&Family=Cable Line', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Titanium Alloy&Family=Cable Line', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Personal&Family=Comm Unit', 'https://www.aonsrd.com/TechItems.aspx?ItemName=System-Wide&Family=Comm Unit', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Unlimited&Family=Comm Unit', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Simple&Family=Lock', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Average&Family=Lock', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Good&Family=Lock', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Superior&Family=Lock', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Medpatch&Family=None', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Basic&Family=Medkit', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Sprayflesh&Family=None', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Advanced&Family=Medkit', 'https://www.aonsrd.com/TechItems.a

Cable Line|Industrial-strength cabling is typically woven from hundreds of cords of high-durability plastic that are often coated in metal for added strength, usually adamantine or titanium alloy.| Cable Line, Adamantine Alloy|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 500/10 ft.|Hands| —; |Bulk| L/20 ft.| Cable Line, Titanium Alloy|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 5/10 ft.|Hands| —; |Bulk| L/10 ft.
['Name', 'Cable Line, Adamantine Alloy', 'Source', 'Starfinder Core Rulebook pg. 219', 'Level', ' 1; ', 'Price', ' 500/10 ft.', 'Hands', ' —; ', 'Description', 'Cable Line|Industrial-strength cabling is typically woven from hundreds of cords of high-durability plastic that are often coated in metal for added strength, usually adamantine or titanium alloy.| ', 'Nethys', 'https://www.aonsrd.com/TechItems.aspx?ItemName=Adamantine Alloy&Family=Cable Line']
Cable Line, Adamantine Alloy
TechItems Cable Line Titanium Alloy
Cable Line|Industrial-strength 

Lock|Locks are extremely advanced reusable devices, able to secure any fastening or closure by bonding with its surface and then holding fast. Locks usually open with a code, security chip, or voice command. Successful Engineering checks can disable locks. Locks come in four levels of quality—simple, average, good, and superior—which determine the DC of Engineering checks to open them without the proper key or code. A lock takes two hands to secure or open with a security chip. For more information on locks, including the Engineering DCs to open them, see the Engineering skill on page 141.| Lock, Simple|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 100|Hands| 2; |Bulk| L| Lock, Average|Source| |Starfinder Core Rulebook pg. 219|Level| 3; |Price| 1,000|Hands| 2; |Bulk| L| Lock, Good|Source| |Starfinder Core Rulebook pg. 219|Level| 6; |Price| 3,600|Hands| 2; |Bulk| L| Lock, Superior|Source| |Starfinder Core Rulebook pg. 219|Level| 14; |Price| 60,000|Hands| 2; |Bulk| L
['Name

Medkit|Medical Gear|Medical gear allows you to attempt Medicine checks and determines the DC of Medicine checks to treat deadly wounds. See the Medicine skill on page 143 for more information.| Medkit, Basic|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 100|Hands| 2; |Bulk| 1|Description|The basic medkit has spray antiseptic, bandages, and handheld instruments designed to examine, explore, and treat common ailments. The basic medkit allows you to attempt DC 25 Medicine checks to treat deadly wounds.| Medkit, Advanced|Source| |Starfinder Core Rulebook pg. 219|Level| 5; |Price| 2,700|Hands| 2; |Bulk| 1|Description|The advanced medkit adds more-advanced and specialized tools to diagnose and treat ailments. The advanced medkit can be used in the same way as a basic medkit, but the DC to treat deadly wounds is only 20, and you gain a +2 insight bonus to Medicine checks to treat drugs, diseases, and poisons. Additionally, you can use an advanced medkit to set up a temporary one

Message Slime|Source| |Starfinder #29: The Cradle Infestation pg. 53|Level| 1; |Price| 50|Hands| —; |Bulk| —|Description|Message slime is a transparent, programmable ooze that comes in an egg-shaped container and is used to deliver clandestine or surprise messages. Any creature with a personal comm unit or other computer can use it to program a message slime with a message up to 5 minutes in length—including both audio and visual components—and requirements that activate the slime’s message. The criteria for activating the message must be simple and detectable with vision or hearing. For example, the slime might be programmed to activate as soon as a creature comes within 30 feet, or as soon as a shirren says the word “hello.” The slime is then spread on a surface, such as a wall, countertop, or door. As soon as the activating conditions are met, the slime’s surface displays its message. Message slime that has not yet been activated can be detected with a successful DC 20 Perception ch

...

Portable Light|A portable light, sometimes called an electric torch, is one of several different devices that use batteries to create light sources. A portable light increases the light level by one step in an area determined by its model, as follows: flashlight (20- foot cone), lantern (10-foot radius), beacon (50-foot radius), and spotlight (100-foot cone).| Portable Light, Beacon|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 25|Capacity| 10; |Usage| 1/hour|Hands| 2; |Bulk| 1| Portable Light, Flashlight|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 1|Capacity| 10; |Usage| 1/hour|Hands| 1; |Bulk| L| Portable Light, Lantern|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 1|Capacity| 10; |Usage| 1/hour|Hands| 1; |Bulk| L| Portable Light, Spotlight|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 25|Capacity| 10; |Usage| 1/hour|Hands| 2; |Bulk| 1
['Name', 'Portable Light, Beacon', 'Source', 'Starfinder Core Rulebook pg. 219', 'Level', ' 

Relic|These rare items are bits of lost technology or unique items less powerful than artifacts. A relic has an item level but can be sold for 100% of the itemâ€™s price (like trade goods). A relic cannot be crafted without the means of a specific formula, which is almost always long lost, and often requires specific materials. A relic that became understood well enough to be reproduced, standardized, and mass-marketed might lose its relic status. |Holdall Raiment|Source| |Starfinder #5: The Thirteenth Gate pg. 40|Level| 4; |Price| 2,000|Hands| —; |Bulk| L|Description|Though this ivory-white frock is tailored for kishalee physiology, most humanoids can wear it with little to no discomfort. A holdall raiment features a dozen pockets, each of which can access a series of linked extradimensional pockets. Each pocket can hold a single item that weighs up to 1 bulk, though any item stored within the raiment can be retrieved from any pocket as a move action, but only by the wearer of the rai

Kishalee Battery|Roughly the same size as standard batteries, kishalee batteries are translucent cylindrical objects bound by a transparent aluminum alloy. By a strange quirk of fate and parallel engineering, kishalee batteries can be interchanged with the standard batteries used among the Pact Worlds and beyond, but these rare batteries allow a creature to use and kishalee artifacts with greater efficiency. If placed in non-kishalee items, a kishalee battery functions like a standard battery. The technology for rapid recharging of a kishalee battery has been lost, but such batteries recharge themselves automatically over the course of 24 hours of disuse.| These batteries come in four different varieties: normal, high-capacity, super-capacity, and ultra-capacity. Each variety has a maximum number of charges as the normal batteries of the same type (|Starfinder Core Rulebook| 179). When these batteries are found, they are almost always fully charged.|Relic|These rare items are bits of l

Kishalee Battery|Roughly the same size as standard batteries, kishalee batteries are translucent cylindrical objects bound by a transparent aluminum alloy. By a strange quirk of fate and parallel engineering, kishalee batteries can be interchanged with the standard batteries used among the Pact Worlds and beyond, but these rare batteries allow a creature to use and kishalee artifacts with greater efficiency. If placed in non-kishalee items, a kishalee battery functions like a standard battery. The technology for rapid recharging of a kishalee battery has been lost, but such batteries recharge themselves automatically over the course of 24 hours of disuse.| These batteries come in four different varieties: normal, high-capacity, super-capacity, and ultra-capacity. Each variety has a maximum number of charges as the normal batteries of the same type (|Starfinder Core Rulebook| 179). When these batteries are found, they are almost always fully charged.|Relic|These rare items are bits of l

Restraints|Binders are simple, single-use, resin-cord restraints that can be placed only on helpless, pinned, or willing creatures (a standard action). A pair of binders immobilizes two limbs of your choice. A creature can escape binders with a successful Acrobatics check (DC = 20 + 1-1/2 Ã— your level).| Manacles are reusable wrist restraints made of hardened metal or ceramics and keyed to open with a security chip, code, or voice command (determined when they are manufactured). Manacles can be placed only on helpless, pinned, or willing creatures (a standard action). Manacles prevent two arms from holding or using any object. A creature can escape manacles with a successful DC 30 Acrobatics check.| Restraints, Binders|Source| |Starfinder Core Rulebook pg. 219|Level| 1; |Price| 5|Hands| —; |Bulk| —| Restraints, Manacles|Source| |Starfinder Core Rulebook pg. 219|Level| 4; |Price| 350|Hands| —; |Bulk| 1
['Name', 'Restraints, Binders', 'Source', 'Starfinder Core Rulebook pg. 219', 'Level

Other Items

In [220]:
link = 'https://aonsrd.com/OtherItems.aspx?Category=Medicinals'
link_to_csv(link, 6, 'Medicinal_Items')

['https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Analgesic', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Antibiotic', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Antiemetic', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Antitoxin', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Coagulant', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Counteractant', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Excitant', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Sal Volatile', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Sedative', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Stimulant', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 1&Family=Vaccine', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 2&Family=Analgesic', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Tier 2

In [221]:
link = 'https://aonsrd.com/OtherItems.aspx?Category=Personal Items'
link_to_csv(link, 6, 'Personal_Items')

['https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Stomach Tabs&Family=None', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Stylemander&Family=None', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Grooming Moss&Family=None', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Spidersatchel&Family=None', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Consumer&Family=Backpack', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Industrial&Family=Backpack', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Athletic&Family=Clothing', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Ceremonial&Family=Clothing', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Environmental&Family=Clothing', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Everyday&Family=Clothing', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Formal&Family=Clothing', 'https://aonsrd.com/OtherItemsDisplay.aspx?ItemName=Party&Family=Clothing', 'https://aonsrd.com/OtherItemsDisplay.aspx?Ite

Space Suit
OtherItemsDisplay None Tool, Manual
Tool, Manual
OtherItemsDisplay Book Holy Text
Book, Holy Text
OtherItemsDisplay Book Standard
Book, Standard
OtherItemsDisplay Riding Saddle Huge or larger
Riding Saddle, Huge or larger
OtherItemsDisplay Riding Saddle Large
Riding Saddle, Large
OtherItemsDisplay Riding Saddle Medium
Riding Saddle, Medium
OtherItemsDisplay Riding Saddle Small or smaller
Riding Saddle, Small or smaller
OtherItemsDisplay None Wake-up Patch
Wake-up Patch
OtherItemsDisplay None Warming Balm
Warming Balm
OtherItemsDisplay None Library Chip
Library Chip
OtherItemsDisplay None Frictionless Gel
Frictionless Gel
OtherItemsDisplay Perfume Standard
Brute fishing:
Perfume, Standard
OtherItemsDisplay Perfume Pheromonal
Brute fishing:
Perfume, Pheromonal
OtherItemsDisplay Tent Mass Produced
Brute fishing:
Tent, Mass Produced
OtherItemsDisplay Tent Mobile Hotelier
Brute fishing:
Tent, Mobile Hotelier
OtherItemsDisplay Umbrella Autoshade
Brute fishing:
Umbrella, Autoshade


Feats

In [240]:
link = 'https://aonsrd.com/Feats.aspx'
link_to_csv(link, 5, 'Feats_End')

['https://aonsrd.com/FeatDisplay.aspx?ItemName=Accelerated Recovery', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Adaptive Casting', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Adaptive Fighting', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Adaptive Resistance', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Adaptive Upgrade', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Add Leverage', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Advance Warning', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Advanced Melee Weapon Proficiency', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Agile Casting', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Ambuscade', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Ambush Awareness', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Amplified Glitch', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Antagonize', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Apt Mentor', 'https://aonsrd.com/FeatDisplay.aspx?ItemName=Arm Extensions', 'https://aonsrd.com/FeatDisp

Poison Rejection (NS)
Polymorph Adept
Positive Conduit
Powered Armored Proficiency (Combat)
Profession Mastery
Protective Fur
Protector of Kahlannal
Psychic Insight
Psychic Power
Psychic Sense
Pull the Pin (Combat)
Quick Draw (Combat)
Quicker Trickler
Reality Glimmer
Reflect Projectiles (Combat)
Reject Chains
Relocate Vitals
Retrotech
Ricochet Grenade (Combat)
Sacred Strike (Combat)
Scurry
Sense Assumptions
Shadow Infusion
Shelter Ally (Combat)
Shield Block (Combat)
Shield Proficiency (Combat)
Shot on the Run (Combat)
Sidestep (Combat)
Skill Focus
Skill Synergy
Sky Jockey
Slam Down (Combat)
Slippery Shooter (Combat)
Small Arm Proficiency (Combat)
Sniper Weapon Proficiency (Combat)
Solid Stance (Combat)
Soothing Telepathy
Special Weapon Proficiency (Combat)
Spell Feint
Spell Focus
Spell Penetration
Spellbane
Spellguard
Spot of Luck
Spring Attack (Combat)
Spry Cover (Combat)
Squox Companion
Stage Magic
Stand Still (Combat)
Startled Scream
Steady Footing
Steel Nerves
Step Up (Combat)
Step